In [ ]:
import io
import os
import json
import pandas as pd
import numpy as np
import random
import distutils.dir_util
from matplotlib import pyplot as plt

In [ ]:
import pandas as pd
playlists = pd.read_json('../0_data/playlists.json', typ = 'frame', encoding='utf-8')
song_meta = pd.read_json('../0_data/song_meta.json', typ = 'frame', encoding='utf-8') 
genre_gn_all = pd.read_json('../0_data/genre_gn_all.json', typ = 'frame', encoding='utf-8', orient='index')

> 플레이리스트 좋아요 수 

In [ ]:
playlists['like_cnt'].describe()

In [ ]:
# 전체 Histogram
plt.hist(playlists['like_cnt'])
plt.show()

=> 좋아요 수의 편차가 심하다.

In [ ]:
# 75%~ Histogram
playlists['like_cnt'].plot.hist(bins=range(2, 29, 1))
plt.show()

In [ ]:
# ~75% Histogram
playlists['like_cnt'].plot.hist(bins=range(30, 53211, 10000))
plt.show()

=> 좋아요 수 25% 이하는 사람들이 사용하는 플레이리스트라고 해석하기 어렵기 때문에 해당 플레이리스트를 삭제한다.

> 결론
1. like_cnt > 5만 남기기

---